In [1]:
include("../../hlt/halite.jl")
include("../salboai.jl")

using Test

H.setdefaultconstants()
g = Salboai.dummyGameMap((32, 32));

In [2]:
forbidden = H.WrappedMatrix(falses(size(g.halite)));

In [16]:
forbidden[CartesianIndex(1,1) + CartesianIndex(-1,0)] = true
forbidden

32×32 Main.H.WrappedArray{Bool,2,BitArray{2}}:
 false  false  false  false  false  …  false  false  false  false   true
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
    

In [17]:
a=copy(forbidden)

32×32 Array{Bool,2}:
 false  false  false  false  false  …  false  false  false  false   true
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
     ⋮                        

In [13]:
forbidden[CartesianIndex(32,1)]

true

In [16]:
allplayerids = [g.players[i].id for i=0:length(g.players)-1]
myplayerid = g.my_player_id
otherplayersids=allplayerids[allplayerids.!=myplayerid]

3-element Array{Int64,1}:
 0
 2
 3

In [12]:
allplayerids = [g.players[i].id for i=1:length(g.players)]
#myplayerid = g.my_player_id
#otherplayersids=allplayerids[allplayerids.!=myplayerid]

KeyError: KeyError: key 4 not found

In [2]:
myplayerid

2

In [17]:
function forbiddensquares(g)
    #forbid certain squares with respect to ENEMY ships (own ship avoidance is handled in avoidcollison)
    #return boolean wrappedmatrix with all squares with enemy ships on them plus/minus one marked as true
    NPlayers=length(g.players)
    myplayerid = g.my_player_id
    allplayerids = [g.players[i].id for i=0:length(g.players)-1]
    otherplayersids=allplayerids[allplayerids.!=myplayerid]

    forbidden = H.WrappedMatrix(falses(size(g.halite)));

    for id = otherplayersids
        Nships = length(g.players[id].ships)
        enemyshippositions = [g.players[id].ships[i].p for i=1:Nships]

        forbidden[enemyshippositions] .= true
        forbidden[enemyshippositions .+ tuple(CartesianIndex(1,0))] .= true
        forbidden[enemyshippositions .+ tuple(CartesianIndex(0,1))] .= true
        forbidden[enemyshippositions .+ tuple(CartesianIndex(-1,0))] .= true
        forbidden[enemyshippositions .+ tuple(CartesianIndex(0,-1))] .= true
    end
    
    #Dont forbid dropoffs even if enemy is near it
    p_dropoffs = H.dropoffs_p(H.me(g))
    for i =1:length(p_dropoffs)
        forbidden[p_dropoffs[i]] .= false
        forbidden[p_dropoffs[i] .+ tuple(CartesianIndex(1,0))] .= false
        forbidden[p_dropoffs[i] .+ tuple(CartesianIndex(0,1))] .= false
        forbidden[p_dropoffs[i] .+ tuple(CartesianIndex(-1,0))] .= false
        forbidden[p_dropoffs[i] .+ tuple(CartesianIndex(0,-1))] .= false
    end
    
    return forbidden
end

forbiddensquares (generic function with 1 method)

In [18]:
forbiddensquares(g)

32×32 Main.H.WrappedArray{Bool,2,BitArray{2}}:
  true   true   true  false  false  …  false  false  false  false  false
 false   true  false  false  false     false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false   true   true   true     false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false      true  false  false  false  false
 false  false  false  false  false  …   true   true  false  false  false
 false  false  false  false  false      true  false  false  false   true
  true  false  false  false  false     false  false  false   true   true
    

In [5]:
[g.players[i].id for i=1:length(g.players)]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [22]:
H.dropoffs_p(H.me(g))

11-element Array{CartesianIndex{2},1}:
 CartesianIndex(17, 6) 
 CartesianIndex(23, 14)
 CartesianIndex(32, 4) 
 CartesianIndex(8, 15) 
 CartesianIndex(22, 30)
 CartesianIndex(5, 21) 
 CartesianIndex(20, 12)
 CartesianIndex(30, 26)
 CartesianIndex(27, 8) 
 CartesianIndex(21, 25)
 CartesianIndex(2, 1)  

In [23]:
H.me(g).shipyard

CartesianIndex(17, 6)